# Get started with primitives

<LegacyContent>
<Admonition type="note">
* This documentation is relevant to IBM Quantum&reg; Platform Classic. If you need the newer version, go to the new [IBM Quantum Platform documentation.](https://quantum.cloud.ibm.com/docs/guides/get-started-with-primitives)
 * While this documentation uses the primitives from Qiskit Runtime, which allow you to use IBM&reg; backends, the primitives can be run on any provider by using the [backend primitives](#backend) instead.  Additionally, you can use the *reference* primitives to run on a local statevector simulator.  See [Exact simulation with Qiskit primitives](/guides/simulate-with-qiskit-sdk-primitives) for details.
</Admonition>
</LegacyContent>
<CloudContent>
<Admonition type="note">
* This documentation is relevant to the new IBM Quantum&reg; Platform. If you need the previous version, return to the [IBM Quantum Platform Classic documentation.](https://docs.quantum.ibm.com/guides/get-started-with-primitives)
* While this documentation uses the primitives from Qiskit Runtime, which allow you to use IBM&reg; backends, the primitives can be run on any provider by using the [backend primitives](#backend) instead.  Additionally, you can use the *reference* primitives to run on a local statevector simulator.  See [Exact simulation with Qiskit primitives](/guides/simulate-with-qiskit-sdk-primitives) for details.
</Admonition>
</CloudContent>

The steps in this topic describe how to set up primitives, explore the options you can use to configure them, and invoke them in a program.

<Admonition type="note" title="Using Fractional Gates">
    To use the newly supported [fractional gates](./fractional-gates), set `use_fractional_gates=True` when requesting a backend from a `QiskitRuntimeService` instance. For example:
    ```python
    service = QiskitRuntimeService()
    fractional_gate_backend = service.least_busy(use_fractional_gates=True)
    ```

    Note that this is an experimental feature and might change in the future.
</Admonition>

<span id="start-estimator"></span>
## Get started with Estimator

{/*Verified the v2 examples 2/29/24 - updated 10/29/24*/}

### 1. Initialize the account

Because Qiskit Runtime Estimator is a managed service, you first need to initialize your account. You can then select the QPU you want to use to calculate the expectation value.

Follow the steps in the [Install and set up topic](install-qiskit) if you don't already have an account.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)

print(backend.name)

### 2. Create a circuit and an observable

You need at least one circuit and one observable as inputs to the Estimator primitive.

In [ ]:
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp

entanglement = [tuple(edge) for edge in backend.coupling_map.get_edges()]
observable = SparsePauliOp.from_sparse_list(
    [("ZZ", [i, j], 0.5) for i, j in entanglement],
    num_qubits=backend.num_qubits,
)
circuit = QAOAAnsatz(observable, reps=2)
# the circuit is parametrized, so we will define the parameter values for execution
param_values = [0.1, 0.2, 0.3, 0.4]

print(f">>> Observable: {observable.paulis}")

The circuit and observable need to be transformed to only use instructions supported by the QPU (referred to as *instruction set architecture (ISA)* circuits). We'll use the transpiler to do this.

In [ ]:
from qiskit.transpiler import generate_preset_pass_manager

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
print(f">>> Circuit ops (ISA): {isa_circuit.count_ops()}")

### 3. Initialize Qiskit Runtime Estimator

When you initialize the Estimator, use the `mode` parameter to specify the mode you want it to run in.  Possible values are `batch`, `session`, or `backend` objects for batch, session, and job execution mode, respectively. For more information, see [Introduction to Qiskit Runtime execution modes.](execution-modes)

In [ ]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator

estimator = Estimator(mode=backend)

### 4. Invoke the Estimator and get results

Next, invoke the `run()` method to calculate expectation values for the input circuits and observables. The circuit, observable, and optional parameter value sets are input as *primitive unified bloc* (PUB) tuples.

In [ ]:
job = estimator.run([(isa_circuit, isa_observable, param_values)])
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

In [ ]:
result = job.result()
print(f">>> {result}")
print(f"  > Expectation value: {result[0].data.evs}")
print(f"  > Metadata: {result[0].metadata}")

<span id="start-sampler"></span>
## Get started with Sampler

### 1. Initialize the account

Because Qiskit Runtime Sampler is a managed service, you first need to initialize your account. You can then select the QPU you want to use to calculate the expectation value.

Follow the steps in the [Install and set up topic](install-qiskit) if you don't already have an account set up.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)

### 2. Create a circuit

You need at least one circuit as the input to the Sampler primitive.

In [ ]:
import numpy as np
from qiskit.circuit.library import EfficientSU2

circuit = EfficientSU2(127, entanglement="linear", flatten=True)
circuit.measure_all()
# The circuit is parametrized, so we will define the parameter values for execution
param_values = np.random.rand(circuit.num_parameters)

Use the transpiler to get an ISA circuit.

In [ ]:
from qiskit.transpiler import generate_preset_pass_manager

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
print(f">>> Circuit ops (ISA): {isa_circuit.count_ops()}")

### 3. Initialize the Qiskit Runtime Sampler

When you initialize the Sampler, use the `mode` parameter to specify the mode you want it to run in.  Possible values are `batch`, `session`, or `backend` objects for batch, session, and job execution mode, respectively. For more information, see [Introduction to Qiskit Runtime execution modes.](execution-modes) <CloudContent>Note that Open Plan users cannot submit session jobs.</CloudContent>

In [ ]:
from qiskit_ibm_runtime import SamplerV2 as Sampler

sampler = Sampler(mode=backend)

### 4. Invoke the Sampler and get results

Next, invoke the `run()` method to generate the output. The circuit and optional parameter value sets are input as *primitive unified bloc* (PUB) tuples.

In [ ]:
job = sampler.run([(isa_circuit, param_values)])
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

In [ ]:
result = job.result()

# Get results for the first (and only) PUB
pub_result = result[0]
print(
    f"Counts for the 'meas' output register: {pub_result.data.meas.get_counts()}"
)

<span id="backend"></span>
## Get started with the backend primitives

Unlike provider-specific primitives, backend primitives are generic implementations that can be used with an arbitrary
`backend` object, as long as it implements the [`Backend`](/api/qiskit/qiskit.providers.Backend) interface.

- The Sampler primitive can be run with any provider by using [`qiskit.primitives.BackendSamplerV2`](/api/qiskit/qiskit.primitives.BackendSamplerV2).
- The Estimator primitive can be run with any provider by using [`qiskit.primitives.BackendEstimatorV2`](../api/qiskit/qiskit.primitives.BackendEstimatorV2).

Some providers implement primitives natively.  See the [Qiskit Ecosystem page](https://qiskit.github.io/ecosystem#provider) for details.

### Example: BackendEstimator

```python
from qiskit.primitives import BackendEstimatorV2
from <some_qiskit_provider> import QiskitProvider

provider = QiskitProvider()
backend = provider.get_backend('backend_name')
estimator = BackendEstimatorV2(backend)
```

### Example: BackendSampler

```python
from qiskit.primitives import BackendSamplerV2
from <some_qiskit_provider> import QiskitProvider

provider = QiskitProvider()
backend = provider.get_backend('backend_name')
sampler = BackendSamplerV2(backend)
```

### Similarities and differences between backend and Runtime primitives

- The inputs to and outputs from [`qiskit.primitives.BackendSamplerV2`](/api/qiskit/qiskit.primitives.BackendSamplerV2) and [`qiskit.primitives.BackendEstimatorV2`](../api/qiskit/qiskit.primitives.BackendEstimatorV2)
follow the same PUB format as the primitives in Qiskit Runtime. See [Primitive inputs and outputs](primitive-input-output) for details.
However, there can be differences in the fields of the returned metadata.

- The [`qiskit.primitives.BackendEstimatorV2`](/api/qiskit/qiskit.primitives.BackendEstimatorV2) class offers no measurement or gate error mitigation implementations out-of-the-box, as
backend primitives are designed to run locally in the user's machine.

- The [`qiskit.primitives.BackendSamplerV2`](/api/qiskit/qiskit.primitives.BackendSamplerV2) class requires a backend that supports the `memory` option.

- The backend primitive interfaces expose custom [`SamplerV2`](/api/qiskit/qiskit.primitives.BackendSamplerV2) and [`EstimatorV2`](/api/qiskit/qiskit.primitives.BackendEstimatorV2) `Options` that are different from the Runtime implementations.

## Next steps

<Admonition type="tip" title="Recommendations">
    - Learn how to [test locally](local-testing-mode) before running on quantum computers.
    - Review detailed [primitives examples.](primitives-examples)
    - Practice with primitives by working through the [Cost function lesson](https://learning.quantum.ibm.com/course/variational-algorithm-design/cost-functions#primitives) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](transpile/) section.
    - Try the [Submit pre-transpiled circuits](https://learning.quantum.ibm.com/tutorial/submitting-user-transpiled-circuits-using-primitives) tutorial.
    - Learn how to [use the primitive options.](runtime-options-overview)
    - View the API for [Sampler](/api/qiskit-ibm-runtime/options-sampler-options) and [Estimator](/api/qiskit-ibm-runtime/options-estimator-options) options.
    - Read [Migrate to V2 primitives](/migration-guides/v2-primitives).
</Admonition>